# Task-4

Load the weights of a pre-trained convolutional neural network included in keras, see https://keras.io/applications/ , and use it as feature extraction method to train a linear model or MLP (scikit-learn or keras are fine) on the pets dataset (http://www.robots.ox.ac.uk/~vgg/data/pets/). Achieve at least 70% accuracy. Store the extracted features on disk so we don’t have to recompute them for model selection. we will be working with the 37 class classification task.

## Pet's Dataset:

A 37 category pet dataset with roughly 200 images for each class. The images have a large variations in scale, pose and lighting. All images have an associated ground truth annotation of breed, head ROI, and pixel level trimap segmentation.

In [1]:
# Importing Libraries

import numpy as np
import pandas as pd
from keras import applications
from keras.applications.vgg16 import preprocess_input
import os
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC
from PIL import Image
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

C:\Users\Rahul\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Rahul\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Importing the dataset 

img_name = [i for i in os.listdir("C:\\images\\images\\")if 'jpg' in i] # Taking just jpg files


In [3]:
# setting image locations

img_name_loc = []
for j in img_name:
    tmp = "C:\\images\\images\\"+j
    img_name_loc.append(tmp)

In [4]:
# Loading images

im = [image.load_img(i, target_size=(224, 224)) for i in img_name_loc]

In [5]:
# Converting images in arrays

X = np.array([image.img_to_array(j) for j in im])

In [6]:
X.shape

(900, 224, 224, 3)

In [7]:
# Building the VGG16 network

model = applications.VGG16(include_top=False,
                           weights='imagenet')

In [8]:
# Summary of the model

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [9]:
vgg_weights, vgg_biases = model.layers[1].get_weights()

In [10]:
vgg_weights.shape

(3, 3, 3, 64)

In [11]:
# Extracting features using VGG

X_pre = preprocess_input(X)
features = model.predict(X_pre)

In [12]:
features.shape

(900, 7, 7, 512)

In [13]:
# Reshaping the features

features_ = features.reshape(int(features.shape[0]), -1)

In [14]:
# Classification with logreg

y = np.zeros(int(features.shape[0]), dtype='int')
y[int(features.shape[0]/2):] = 1

In [15]:
X_train, X_test, y_train, y_test = train_test_split(features_, y, stratify=y)

In [16]:
lr = LogisticRegression().fit(X_train, y_train)


In [17]:
print('Train score:',lr.score(X_train, y_train))
print('Test score:',lr.score(X_test, y_test))

Train score: 1.0
Test score: 0.9244444444444444
